In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
import pandas as pd
from pyspark.sql import types as T
from pyspark import SparkFiles
from textblob import TextBlob
import plotly.graph_objects as go
import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)

In [2]:
spark = SparkSession\
        .builder\
        .master("local")\
        .config("spark.files.overwrite", True)\
        .getOrCreate()

In [3]:
def get_complete_boston_data(listings_url, reviews_url, year):
    spark.sparkContext.addFile(reviews_url)
    spark.sparkContext.addFile(listings_url)
    boston_reviews_df = spark.read.option("header", "true").csv(SparkFiles.get("reviews.csv.gz"))
    boston_listings_df = spark.read.csv(SparkFiles.get("listings.csv"), header=True)
    boston_reviews_df = boston_reviews_df.where("comments != 'None'")
    boston_reviews_df = boston_reviews_df.withColumnRenamed("id", "review_id")
    boston_top_neighbourhood = boston_listings_df.join(boston_neighbourhoods_df, ["neighbourhood"] , "left_semi")
    boston_top_neighbourhood = boston_top_neighbourhood.join(boston_reviews_df, boston_top_neighbourhood["id"] == boston_reviews_df["listing_id"], "left")
    return boston_top_neighbourhood.withColumn("year", F.lit(year))

In [4]:
neighbourhoods_list = ['Fenway', 'Allston', 'Dorchester']
boston_neighbourhoods_df = spark.createDataFrame(neighbourhoods_list, StringType())
boston_neighbourhoods_df = boston_neighbourhoods_df.selectExpr("value as neighbourhood")

In [5]:
reviews_url = "http://data.insideairbnb.com/united-states/ma/boston/2020-06-10/data/reviews.csv.gz"
listings_url = "http://data.insideairbnb.com/united-states/ma/boston/2020-06-10/visualisations/listings.csv"

boston_neighbourhood_2020 = get_complete_boston_data(listings_url, reviews_url, 2020)
boston_neighbourhood_2020.toPandas()

,neighbourhood,id,name,host_id,host_name,neighbourhood_group,latitude,longitude,room_type,price,...,reviews_per_month,calculated_host_listings_count,availability_365,listing_id,review_id,date,reviewer_id,reviewer_name,comments,year
0,Dorchester,22571244,✪2 Double beds 4mi to Downtown near subway fre...,25203946,Eder,None,42.30936,-71.06347,Private room,44,...,0.34,17,160,22571244,228392128,2018-01-19,162953953,Samuel,Really a nice stay. Warm accommodations. A lot...,2020
1,Dorchester,22571244,✪2 Double beds 4mi to Downtown near subway fre...,25203946,Eder,None,42.30936,-71.06347,Private room,44,...,0.34,17,160,22571244,232850966,2018-02-06,108441156,Loise Coleen,This place is a wonderful value in a very nice...,2020
2,Dorchester,22571244,✪2 Double beds 4mi to Downtown near subway fre...,25203946,Eder,None,42.30936,-71.06347,Private room,44,...,0.34,17,160,22571244,249045170,2018-04-01,50884175,Marcelo,"Eder, is the best host I had so far! He’s amaz...",2020
3,Dorchester,22571244,✪2 Double beds 4mi to Downtown near subway fre...,25203946,Eder,None,42.30936,-71.06347,Private room,44,...,0.34,17,160,22571244,276689352,2018-06-14,126237644,Gloria,La casa è molto confortevole e il marito di E...,2020
4,Dorchester,22571244,✪2 Double beds 4mi to Downtown near subway fre...,25203946,Eder,None,42.30936,-71.06347,Private room,44,...,0.34,17,160,22571244,279053437,2018-06-19,140123458,F,"Located close to Boston, this place is a home ...",2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29224,Allston,38131632,"Gorgeous space in vibrant area, steps to the t614",18110188,Cindy,None,42.35192,-71.12404,Entire home/apt,133,...,3.71,48,78,38131632,614850845,2020-03-06,65276350,Patty,Es un apartamento,2020
29225,Allston,38131632,"Gorgeous space in vibrant area, steps to the t614",18110188,Cindy,None,42.35192,-71.12404,Entire home/apt,133,...,3.71,48,78,38131632,616753381,2020-03-10,45242217,A&J,Great space. Near many restaurants but also qu...,2020
29226,Allston,38131632,"Gorgeous space in vibrant area, steps to the t614",18110188,Cindy,None,42.35192,-71.12404,Entire home/apt,133,...,3.71,48,78,38131632,617688297,2020-03-13,153288282,Rowan,Lovely stay! Definitely a great place for food...,2020
29227,Allston,38131632,"Gorgeous space in vibrant area, steps to the t614",18110188,Cindy,None,42.35192,-71.12404,Entire home/apt,133,...,3.71,48,78,38131632,618083877,2020-03-14,313838678,Alisha,Good place for value,2020


In [6]:
reviews_url = "http://data.insideairbnb.com/united-states/ma/boston/2019-06-14/data/reviews.csv.gz"
listings_url = "http://data.insideairbnb.com/united-states/ma/boston/2019-06-14/visualisations/listings.csv"
boston_neighbourhood_2019 = get_complete_boston_data(listings_url, reviews_url, 2019)
boston_neighbourhood_2019.toPandas()

,neighbourhood,id,name,host_id,host_name,neighbourhood_group,latitude,longitude,room_type,price,...,reviews_per_month,calculated_host_listings_count,availability_365,listing_id,review_id,date,reviewer_id,reviewer_name,comments,year
0,Dorchester,17063804,Jacques & Smriti's large bright room by JFK-UMASS,57812241,Alexia,None,42.31838,-71.05671,Private room,75,...,0.20,7,219,17063804,152646064,2017-05-17,85499345,Smriti,Alexia is an amazing host and such a wonderful...,2019
1,Dorchester,17063804,Jacques & Smriti's large bright room by JFK-UMASS,57812241,Alexia,None,42.31838,-71.05671,Private room,75,...,0.20,7,219,17063804,154022197,2017-05-22,52183481,Florin,"Alexia is very friendly, good knowledge of the...",2019
2,Dorchester,17063804,Jacques & Smriti's large bright room by JFK-UMASS,57812241,Alexia,None,42.31838,-71.05671,Private room,75,...,0.20,7,219,17063804,213046934,2017-11-19,156382082,Tamaki,Located within 15-minute-walk distance from JF...,2019
3,Dorchester,17063804,Jacques & Smriti's large bright room by JFK-UMASS,57812241,Alexia,None,42.31838,-71.05671,Private room,75,...,0.20,7,219,17063804,254925324,2018-04-18,40103131,Haiko & Fleur,Without doubt Alexia is the best airbnb host w...,2019
4,Dorchester,17063804,Jacques & Smriti's large bright room by JFK-UMASS,57812241,Alexia,None,42.31838,-71.05671,Private room,75,...,0.20,7,219,17063804,264698249,2018-05-14,86247216,Mine,Great location: a short walk to the red line s...,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39882,Dorchester,26138090,Comfortable room in Dorchester (#1),188413439,Hilson,None,42.3065,-71.06666,Private room,65,...,2.91,4,71,26138090,467659647,2019-06-10,221753343,Rohan,"Air bnb itself is fine, area around not so good.",2019
39883,Allston,27991699,Boston Spot,211416849,Adam,None,42.35221,-71.14307,Private room,115,...,None,1,364,None,None,None,None,None,None,2019
39884,Dorchester,35238309,Location location location and so beautiful!!!...,261646616,Joseph,None,42.31584,-71.05796,Private room,500,...,None,3,85,None,None,None,None,None,None,2019
39885,Allston,35614764,Spacious 1 bedroom apartment on commonwealth ave,90175388,Alekya,None,42.35299,-71.12243,Entire home/apt,200,...,None,1,25,None,None,None,None,None,None,2019


In [7]:
reviews_url = "http://data.insideairbnb.com/united-states/ma/boston/2018-07-18/data/reviews.csv.gz"
listings_url = "http://data.insideairbnb.com/united-states/ma/boston/2018-07-18/visualisations/listings.csv"
boston_neighbourhood_2018 = get_complete_boston_data(listings_url, reviews_url, 2018)
boston_neighbourhood_2018.toPandas()

,neighbourhood,id,name,host_id,host_name,neighbourhood_group,latitude,longitude,room_type,price,...,reviews_per_month,calculated_host_listings_count,availability_365,listing_id,review_id,date,reviewer_id,reviewer_name,comments,year
0,Dorchester,17063804,Jacques & Smriti's large bright room by JFK-UMASS,57812241,Alexia,None,42.31837726494946,-71.05671457362139,Private room,75,...,0.35,7,243,17063804,152646064,2017-05-17,85499345,Smriti,Alexia is an amazing host and such a wonderful...,2018
1,Dorchester,17063804,Jacques & Smriti's large bright room by JFK-UMASS,57812241,Alexia,None,42.31837726494946,-71.05671457362139,Private room,75,...,0.35,7,243,17063804,154022197,2017-05-22,52183481,Florin,"Alexia is very friendly, good knowledge of the...",2018
2,Dorchester,17063804,Jacques & Smriti's large bright room by JFK-UMASS,57812241,Alexia,None,42.31837726494946,-71.05671457362139,Private room,75,...,0.35,7,243,17063804,213046934,2017-11-19,156382082,Tamaki,Located within 15-minute-walk distance from JF...,2018
3,Dorchester,17063804,Jacques & Smriti's large bright room by JFK-UMASS,57812241,Alexia,None,42.31837726494946,-71.05671457362139,Private room,75,...,0.35,7,243,17063804,254925324,2018-04-18,40103131,Haiko & Fleur,Without doubt Alexia is the best airbnb host w...,2018
4,Dorchester,17063804,Jacques & Smriti's large bright room by JFK-UMASS,57812241,Alexia,None,42.31837726494946,-71.05671457362139,Private room,75,...,0.35,7,243,17063804,264698249,2018-05-14,86247216,Fikret,Great location: a short walk to the red line s...,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31699,Allston,25914450,Lovely Day,187707739,Yehuda,None,42.35375808904245,-71.13733178267444,Private room,65,...,2.81,6,54,25914450,279761091,2018-06-21,63818801,Daniel,Nice and clean space! The location is convenie...,2018
31700,Allston,25914450,Lovely Day,187707739,Yehuda,None,42.35375808904245,-71.13733178267444,Private room,65,...,2.81,6,54,25914450,281828334,2018-06-25,28588628,Aditi,The place is very well located and is incredib...,2018
31701,Dorchester,26138090,Comfortable room in Dorchester (#1),188413439,Hilson,None,42.306501109039026,-71.06665534423358,Private room,65,...,1,4,33,26138090,288729383,2018-07-10,179467430,LaDoris,"This is a great place to stay!!! Very clean, q...",2018
31702,Fenway,26935855,Amazing view of the apartme,194322480,Nhgio,None,42.34572531468303,-71.08824820401912,Private room,204,...,None,1,362,None,None,None,None,None,None,2018


In [8]:
reviews_url = "http://data.insideairbnb.com/united-states/ma/boston/2017-10-06/data/reviews.csv.gz"
listings_url = "http://data.insideairbnb.com/united-states/ma/boston/2017-10-06/visualisations/listings.csv"
boston_neighbourhood_2017 = get_complete_boston_data(listings_url, reviews_url, 2017)
boston_neighbourhood_2017.toPandas()

,neighbourhood,id,name,host_id,host_name,neighbourhood_group,latitude,longitude,room_type,price,...,reviews_per_month,calculated_host_listings_count,availability_365,listing_id,review_id,date,reviewer_id,reviewer_name,comments,year
0,Dorchester,10743879,"Charming, sun filled condo in quiet neighborhood",41269443,Allison,None,42.3172840909121,-71.05493708945174,Entire home/apt,150,...,1.19,1,22,10743879,113687242,2016-11-13,103088025,Gerri,We had a great stay at Allison's home. Her pl...,2017
1,Dorchester,10743879,"Charming, sun filled condo in quiet neighborhood",41269443,Allison,None,42.3172840909121,-71.05493708945174,Entire home/apt,150,...,1.19,1,22,10743879,121702696,2016-12-18,105866458,Jennifer,Staying in Allison's home was such a wonderful...,2017
2,Dorchester,10743879,"Charming, sun filled condo in quiet neighborhood",41269443,Allison,None,42.3172840909121,-71.05493708945174,Entire home/apt,150,...,1.19,1,22,10743879,132845450,2017-02-19,116909212,Brenna,Allisons place was perfect! Especially if you ...,2017
3,Dorchester,10743879,"Charming, sun filled condo in quiet neighborhood",41269443,Allison,None,42.3172840909121,-71.05493708945174,Entire home/apt,150,...,1.19,1,22,10743879,136863198,2017-03-12,118336543,Peter,"""This place was amazing, I'd even go as far as...",2017
4,Dorchester,10743879,"Charming, sun filled condo in quiet neighborhood",41269443,Allison,None,42.3172840909121,-71.05493708945174,Entire home/apt,150,...,1.19,1,22,10743879,138393020,2017-03-19,110442436,Nicole,The house was a little out of the way for the ...,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20902,Dorchester,19599031,Home away from home,118883513,Darryl,None,42.28615614997633,-71.03953104938581,Private room,70,...,7.32,1,347,19599031,197092669,2017-09-24,150881862,Dione,"Amazing host, place was lovely and would recom...",2017
20903,Dorchester,19599031,Home away from home,118883513,Darryl,None,42.28615614997633,-71.03953104938581,Private room,70,...,7.32,1,347,19599031,198802427,2017-09-30,152268461,David,Darryl was a great host (and so was Gigi!). Th...,2017
20904,Dorchester,21148820,Private Apartment Close to Redline T Subway,18479599,Agnieszka,None,42.325976820904145,-71.06016439244476,Entire home/apt,250,...,None,9,309,None,None,None,None,None,None,2017
20905,Dorchester,21217710,Cozy Boston Room,153299935,NhuanKim,None,42.31497905682507,-71.0564388777616,Private room,35,...,None,1,56,None,None,None,None,None,None,2017


In [9]:
reviews_url = "http://data.insideairbnb.com/united-states/ma/boston/2016-09-07/data/reviews.csv.gz"
listings_url = "http://data.insideairbnb.com/united-states/ma/boston/2016-09-07/visualisations/listings.csv"
boston_neighbourhood_2016 = get_complete_boston_data(listings_url, reviews_url, 2016)
boston_neighbourhood_2016.toPandas()

,neighbourhood,id,name,host_id,host_name,neighbourhood_group,latitude,longitude,room_type,price,...,reviews_per_month,calculated_host_listings_count,availability_365,listing_id,review_id,date,reviewer_id,reviewer_name,comments,year
0,Allston,4402209,l GREAT ROOM BOSOTN,8139434,Ari,None,42.35208077328276,-71.13599057398088,Private room,100,...,3.16,10,119,4402209,98811254,2016-09-02,73127997,紹桀,Great( ´` )ﾉ,2016
1,Allston,4402209,l GREAT ROOM BOSOTN,8139434,Ari,None,42.35208077328276,-71.13599057398088,Private room,100,...,3.16,10,119,4402209,98653881,2016-09-01,73127997,紹桀,"Ari is a really nice host, willing to help me ...",2016
2,Allston,4402209,l GREAT ROOM BOSOTN,8139434,Ari,None,42.35208077328276,-71.13599057398088,Private room,100,...,3.16,10,119,4402209,89696317,2016-07-29,18169449,Cui,"It's a very convenient location, and Ari provi...",2016
3,Allston,4402209,l GREAT ROOM BOSOTN,8139434,Ari,None,42.35208077328276,-71.13599057398088,Private room,100,...,3.16,10,119,4402209,82408950,2016-06-27,62950686,Atalya,Ari has this hosting thing down to a T! Pure p...,2016
4,Allston,4402209,l GREAT ROOM BOSOTN,8139434,Ari,None,42.35208077328276,-71.13599057398088,Private room,100,...,3.16,10,119,4402209,82051490,2016-06-26,61663856,Yechiel,Just like every time my stay was perfect:-),2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13385,Fenway,3589267,There's No Place Like {Our} Home,11890037,Will & Mari,None,42.34122489190963,-71.0943622960716,Entire home/apt,699,...,0.62,1,343,3589267,18252118,2014-08-24,16092941,Igor,Everything was great! Some food in fridge afte...,2016
13386,Fenway,3589267,There's No Place Like {Our} Home,11890037,Will & Mari,None,42.34122489190963,-71.0943622960716,Entire home/apt,699,...,0.62,1,343,3589267,17118645,2014-08-07,18355131,Kevin,The home was just as described and the neighbo...,2016
13387,Fenway,3589267,There's No Place Like {Our} Home,11890037,Will & Mari,None,42.34122489190963,-71.0943622960716,Entire home/apt,699,...,0.62,1,343,3589267,17062693,2014-08-06,14749343,Jean Louis,"Appartement très bien situé, très près des mus...",2016
13388,Fenway,3589267,There's No Place Like {Our} Home,11890037,Will & Mari,None,42.34122489190963,-71.0943622960716,Entire home/apt,699,...,0.62,1,343,3589267,16856640,2014-08-03,1912165,Caroline,We had a great time in Boston and Mari's place...,2016


In [10]:
reviews_url = "http://data.insideairbnb.com/united-states/ma/boston/2015-10-03/data/reviews.csv.gz"
listings_url = "http://data.insideairbnb.com/united-states/ma/boston/2015-10-03/visualisations/listings.csv"
boston_neighbourhood_2015 = get_complete_boston_data(listings_url, reviews_url, 2015)
boston_neighbourhood_2015.toPandas()

,neighbourhood,id,name,host_id,host_name,neighbourhood_group,latitude,longitude,room_type,price,...,reviews_per_month,calculated_host_listings_count,availability_365,listing_id,review_id,date,reviewer_id,reviewer_name,comments,year
0,Allston,5103430,Brand New duplex - 3 BDR / 3 BA,26367207,Raphael,None,42.3521929684635,-71.12961763207782,Entire home/apt,310,...,0.88,3,68,5103430,44668713,2015-08-28,12066100,Eddie,"The house owner, Raphael, is a very kind and e...",2015
1,Allston,5103430,Brand New duplex - 3 BDR / 3 BA,26367207,Raphael,None,42.3521929684635,-71.12961763207782,Entire home/apt,310,...,0.88,3,68,5103430,35203143,2015-06-16,29246587,Michael,Raphael's home was well maintained and very cl...,2015
2,Allston,5103430,Brand New duplex - 3 BDR / 3 BA,26367207,Raphael,None,42.3521929684635,-71.12961763207782,Entire home/apt,310,...,0.88,3,68,5103430,34523184,2015-06-09,26172960,Joe,"""This apartment was perfect for our trip to a ...",2015
3,Allston,5103430,Brand New duplex - 3 BDR / 3 BA,26367207,Raphael,None,42.3521929684635,-71.12961763207782,Entire home/apt,310,...,0.88,3,68,5103430,32679028,2015-05-20,5961958,Erich,"Awesome place, great stay, host was very helpf...",2015
4,Allston,7052847,1 big room in a cute 2 bedroom apt,10336060,Rebeca,None,42.352805645017064,-71.12398135726829,Private room,60,...,None,1,70,None,None,None,None,None,None,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8475,Fenway,7332266,"Sunny private room,Longwood, Fenway",31340713,Kama,None,42.34534204525907,-71.10843918882692,Private room,85,...,5.00,3,340,7332266,44822969,2015-08-29,8913208,Veronica,"The neighborhood is lovely, and the room too. ...",2015
8476,Fenway,2083261,Beautiful Apart in Cambridgeport,10641213,Sejal,None,42.35465366568941,-71.10868079403137,Private room,78,...,0.59,1,302,2083261,33890486,2015-06-01,34310541,Imane,The host canceled this reservation 5 days befo...,2015
8477,Fenway,2083261,Beautiful Apart in Cambridgeport,10641213,Sejal,None,42.35465366568941,-71.10868079403137,Private room,78,...,0.59,1,302,2083261,30540240,2015-04-23,14969968,Eric,The reservation was canceled 8 days before arr...,2015
8478,Fenway,2083261,Beautiful Apart in Cambridgeport,10641213,Sejal,None,42.35465366568941,-71.10868079403137,Private room,78,...,0.59,1,302,2083261,30540239,2015-04-23,31627912,Shreya,The reservation was canceled 21 days before ar...,2015


Now that we have the combined data for all 5 years for the required neighbourhoods, we can go into a deepdive on each of the neighbourhoods

Fenway

Let's combine Fenway data from all the years

In [11]:
fenway_neighbourhood = boston_neighbourhood_2015.where("neighbourhood == 'Fenway'").union(boston_neighbourhood_2016.where("neighbourhood == 'Fenway'"))
fenway_neighbourhood = fenway_neighbourhood.union(boston_neighbourhood_2017.where("neighbourhood == 'Fenway'"))
fenway_neighbourhood = fenway_neighbourhood.union(boston_neighbourhood_2018.where("neighbourhood == 'Fenway'"))
fenway_neighbourhood = fenway_neighbourhood.union(boston_neighbourhood_2019.where("neighbourhood == 'Fenway'"))
fenway_neighbourhood = fenway_neighbourhood.union(boston_neighbourhood_2020.where("neighbourhood == 'Fenway'"))
fenway_neighbourhood.toPandas()

,neighbourhood,id,name,host_id,host_name,neighbourhood_group,latitude,longitude,room_type,price,...,reviews_per_month,calculated_host_listings_count,availability_365,listing_id,review_id,date,reviewer_id,reviewer_name,comments,year
0,Fenway,8552387,Bright Back Bay Brownstone,28880663,Amanda,None,42.35117407918826,-71.08998038575537,Entire home/apt,225,...,None,1,9,None,None,None,None,None,None,2015
1,Fenway,8469841,Large Room Back Bay Close to MIT,16953664,Ife,None,42.3510130004383,-71.09012784825632,Private room,90,...,1,3,346,8469841,48817108,2015-09-28,44610837,Alan,I had a great 1st experience using airbnb. If...,2015
2,Fenway,7757132,Charming and spacious 1 bedroom,8958021,Lian,None,42.34207635265539,-71.09799981268785,Entire home/apt,190,...,1.71,1,58,7757132,47402256,2015-09-18,4228486,Natalie,"The apartment is clean, comfortable and very ...",2015
3,Fenway,7757132,Charming and spacious 1 bedroom,8958021,Lian,None,42.34207635265539,-71.09799981268785,Entire home/apt,190,...,1.71,1,58,7757132,44984587,2015-08-30,36039829,Marco,I booked Lian's flat for my parents and they r...,2015
4,Fenway,5086521,Modern & spacious studio for two,1868179,Ryan,None,42.34086101708686,-71.08828863604951,Entire home/apt,150,...,1.55,1,130,5086521,46117564,2015-09-08,31812081,Maximilian,"The place was exactly as described, location w...",2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9073,Fenway,5671806,Luxury Fenway Brownstone-Beacon St,29406566,Jennifer,None,42.34535000231358,-71.10684236073557,Entire home/apt,400,...,0.90,1,29,5671806,30315640,2015-04-20,28743839,Nanci,The description of this place is (email hidden...,2020
9074,Fenway,5671806,Luxury Fenway Brownstone-Beacon St,29406566,Jennifer,None,42.34535000231358,-71.10684236073557,Entire home/apt,400,...,0.90,1,29,5671806,32567096,2015-05-19,15747050,Ann,A very lovely place in a perfect location. Ne...,2020
9075,Fenway,5671806,Luxury Fenway Brownstone-Beacon St,29406566,Jennifer,None,42.34535000231358,-71.10684236073557,Entire home/apt,400,...,0.90,1,29,5671806,33313882,2015-05-26,26989637,Prashant,Jennifer was really helpful and provided me an...,2020
9076,Fenway,5671806,Luxury Fenway Brownstone-Beacon St,29406566,Jennifer,None,42.34535000231358,-71.10684236073557,Entire home/apt,400,...,0.90,1,29,5671806,37398353,2015-07-06,33661506,Kathleen,Jennifer and the apartment were great! At firs...,2020


Get the setiment score for each review using textblob library, but before that, the reviews need to be translated to english, as not all the reviews are in english

In [12]:
def translate(sentence):
    try:
        str(TextBlob(x).translate(to= 'en'))
    except:
        return sentence

In [13]:
translate_udf = F.udf(lambda sentence: translate(sentence) if sentence is not None else None , T.StringType())

In [14]:
fenway_neighbourhood = fenway_neighbourhood.withColumn('comments', translate_udf(F.col("comments")))
fenway_neighbourhood.toPandas()

,neighbourhood,id,name,host_id,host_name,neighbourhood_group,latitude,longitude,room_type,price,...,reviews_per_month,calculated_host_listings_count,availability_365,listing_id,review_id,date,reviewer_id,reviewer_name,comments,year
0,Fenway,8552387,Bright Back Bay Brownstone,28880663,Amanda,None,42.35117407918826,-71.08998038575537,Entire home/apt,225,...,None,1,9,None,None,None,None,None,None,2015
1,Fenway,8469841,Large Room Back Bay Close to MIT,16953664,Ife,None,42.3510130004383,-71.09012784825632,Private room,90,...,1,3,346,8469841,48817108,2015-09-28,44610837,Alan,I had a great 1st experience using airbnb. If...,2015
2,Fenway,7757132,Charming and spacious 1 bedroom,8958021,Lian,None,42.34207635265539,-71.09799981268785,Entire home/apt,190,...,1.71,1,58,7757132,47402256,2015-09-18,4228486,Natalie,"The apartment is clean, comfortable and very ...",2015
3,Fenway,7757132,Charming and spacious 1 bedroom,8958021,Lian,None,42.34207635265539,-71.09799981268785,Entire home/apt,190,...,1.71,1,58,7757132,44984587,2015-08-30,36039829,Marco,I booked Lian's flat for my parents and they r...,2015
4,Fenway,5086521,Modern & spacious studio for two,1868179,Ryan,None,42.34086101708686,-71.08828863604951,Entire home/apt,150,...,1.55,1,130,5086521,46117564,2015-09-08,31812081,Maximilian,"The place was exactly as described, location w...",2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9073,Fenway,5671806,Luxury Fenway Brownstone-Beacon St,29406566,Jennifer,None,42.34535000231358,-71.10684236073557,Entire home/apt,400,...,0.90,1,29,5671806,30315640,2015-04-20,28743839,Nanci,The description of this place is (email hidden...,2020
9074,Fenway,5671806,Luxury Fenway Brownstone-Beacon St,29406566,Jennifer,None,42.34535000231358,-71.10684236073557,Entire home/apt,400,...,0.90,1,29,5671806,32567096,2015-05-19,15747050,Ann,A very lovely place in a perfect location. Ne...,2020
9075,Fenway,5671806,Luxury Fenway Brownstone-Beacon St,29406566,Jennifer,None,42.34535000231358,-71.10684236073557,Entire home/apt,400,...,0.90,1,29,5671806,33313882,2015-05-26,26989637,Prashant,Jennifer was really helpful and provided me an...,2020
9076,Fenway,5671806,Luxury Fenway Brownstone-Beacon St,29406566,Jennifer,None,42.34535000231358,-71.10684236073557,Entire home/apt,400,...,0.90,1,29,5671806,37398353,2015-07-06,33661506,Kathleen,Jennifer and the apartment were great! At firs...,2020


Sentiment Score:

In [15]:
sentiment_udf = F.udf(lambda x:TextBlob(x).sentiment.polarity if x is not None else None , T.StringType())

In [16]:
fenway_neighbourhood = fenway_neighbourhood.withColumn('review_polarity', sentiment_udf(F.col("comments")))
fenway_neighbourhood.toPandas()

,neighbourhood,id,name,host_id,host_name,neighbourhood_group,latitude,longitude,room_type,price,...,calculated_host_listings_count,availability_365,listing_id,review_id,date,reviewer_id,reviewer_name,comments,year,review_polarity
0,Fenway,8552387,Bright Back Bay Brownstone,28880663,Amanda,None,42.35117407918826,-71.08998038575537,Entire home/apt,225,...,1,9,None,None,None,None,None,None,2015,None
1,Fenway,8469841,Large Room Back Bay Close to MIT,16953664,Ife,None,42.3510130004383,-71.09012784825632,Private room,90,...,3,346,8469841,48817108,2015-09-28,44610837,Alan,I had a great 1st experience using airbnb. If...,2015,0.31500000000000006
2,Fenway,7757132,Charming and spacious 1 bedroom,8958021,Lian,None,42.34207635265539,-71.09799981268785,Entire home/apt,190,...,1,58,7757132,47402256,2015-09-18,4228486,Natalie,"The apartment is clean, comfortable and very ...",2015,0.3766666666666667
3,Fenway,7757132,Charming and spacious 1 bedroom,8958021,Lian,None,42.34207635265539,-71.09799981268785,Entire home/apt,190,...,1,58,7757132,44984587,2015-08-30,36039829,Marco,I booked Lian's flat for my parents and they r...,2015,0.07714285714285714
4,Fenway,5086521,Modern & spacious studio for two,1868179,Ryan,None,42.34086101708686,-71.08828863604951,Entire home/apt,150,...,1,130,5086521,46117564,2015-09-08,31812081,Maximilian,"The place was exactly as described, location w...",2015,0.4461111111111111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9073,Fenway,5671806,Luxury Fenway Brownstone-Beacon St,29406566,Jennifer,None,42.34535000231358,-71.10684236073557,Entire home/apt,400,...,1,29,5671806,30315640,2015-04-20,28743839,Nanci,The description of this place is (email hidden...,2020,0.3726190476190476
9074,Fenway,5671806,Luxury Fenway Brownstone-Beacon St,29406566,Jennifer,None,42.34535000231358,-71.10684236073557,Entire home/apt,400,...,1,29,5671806,32567096,2015-05-19,15747050,Ann,A very lovely place in a perfect location. Ne...,2020,0.21796969696969692
9075,Fenway,5671806,Luxury Fenway Brownstone-Beacon St,29406566,Jennifer,None,42.34535000231358,-71.10684236073557,Entire home/apt,400,...,1,29,5671806,33313882,2015-05-26,26989637,Prashant,Jennifer was really helpful and provided me an...,2020,0.4979166666666667
9076,Fenway,5671806,Luxury Fenway Brownstone-Beacon St,29406566,Jennifer,None,42.34535000231358,-71.10684236073557,Entire home/apt,400,...,1,29,5671806,37398353,2015-07-06,33661506,Kathleen,Jennifer and the apartment were great! At firs...,2020,0.13065907522429263


In [17]:
fenway_neighbourhood_home = fenway_neighbourhood.where("room_type == 'Entire home/apt'").toPandas()
fig = px.histogram(fenway_neighbourhood_home, x="review_polarity")
fig.show()

In [18]:
fenway_neighbourhood_private = fenway_neighbourhood.where("room_type == 'Private room'").toPandas()
fig = px.histogram(fenway_neighbourhood_private, x="review_polarity")
fig.show()

In [19]:
fenway_neighbourhood_shared = fenway_neighbourhood.where("room_type == 'Shared room'").toPandas()
fig = px.histogram(fenway_neighbourhood_private, x="review_polarity")
fig.show()

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 50556)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.7/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.7/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.7/socketserver.py", line 720, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 269, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/local/spark/python/pyspark/accumulators.py", line 245, in accum_updates
    num_updates = read_int(self.rfile)
  File "/usr/local/spark/python/pysp

Allston:

Let's combine Allston data from all the years

In [21]:
allston_neighbourhood = boston_neighbourhood_2015.where("neighbourhood == 'Allston'").union(boston_neighbourhood_2016.where("neighbourhood == 'Allston'"))
allston_neighbourhood = allston_neighbourhood.union(boston_neighbourhood_2017.where("neighbourhood == 'Allston'"))
allston_neighbourhood = allston_neighbourhood.union(boston_neighbourhood_2018.where("neighbourhood == 'Allston'"))
allston_neighbourhood = allston_neighbourhood.union(boston_neighbourhood_2019.where("neighbourhood == 'Allston'"))
allston_neighbourhood = allston_neighbourhood.union(boston_neighbourhood_2020.where("neighbourhood == 'Allston'"))
allston_neighbourhood.toPandas()

NameError: name 'boston_neighbourhood_2015' is not defined

Let's translate all the reviews to english first

In [22]:
allston_neighbourhood = allston_neighbourhood.withColumn('comments', translate_udf(F.col("comments")))
allston_neighbourhood.toPandas()

NameError: name 'allston_neighbourhood' is not defined

Calculate the polarity of each review

In [23]:
allston_neighbourhood = allston_neighbourhood.withColumn('review_polarity', sentiment_udf(F.col("comments")))
allston_neighbourhood.toPandas()

NameError: name 'allston_neighbourhood' is not defined

In [24]:
allston_neighbourhood_home = allston_neighbourhood.where("room_type == 'Entire home/apt'").toPandas()
fig = px.histogram(allston_neighbourhood_home, x="review_polarity", color='room_type')
fig.show()

NameError: name 'allston_neighbourhood' is not defined

In [25]:
allston_neighbourhood_private = allston_neighbourhood.where("room_type == 'Private room'").toPandas()
fig = px.histogram(allston_neighbourhood_private, x="review_polarity", color='room_type')
fig.show()

NameError: name 'allston_neighbourhood' is not defined

In [26]:
allston_neighbourhood_shared = allston_neighbourhood.where("room_type == 'Shared room'").toPandas()
fig = px.histogram(allston_neighbourhood_shared, x="review_polarity", color='room_type')
fig.show()

NameError: name 'allston_neighbourhood' is not defined

Dorchester:

Let's combine Dorchester data from all the years

In [27]:
dorchester_neighbourhood = boston_neighbourhood_2015.where("neighbourhood == 'Dorchester'").union(boston_neighbourhood_2016.where("neighbourhood == 'Dorchester'"))
dorchester_neighbourhood = dorchester_neighbourhood.union(boston_neighbourhood_2017.where("neighbourhood == 'Dorchester'"))
dorchester_neighbourhood = dorchester_neighbourhood.union(boston_neighbourhood_2018.where("neighbourhood == 'Dorchester'"))
dorchester_neighbourhood = dorchester_neighbourhood.union(boston_neighbourhood_2019.where("neighbourhood == 'Dorchester'"))
dorchester_neighbourhood = dorchester_neighbourhood.union(boston_neighbourhood_2020.where("neighbourhood == 'Dorchester'"))
dorchester_neighbourhood.toPandas()

NameError: name 'boston_neighbourhood_2015' is not defined

In [28]:
dorchester_neighbourhood = dorchester_neighbourhood.withColumn('comments', translate_udf(F.col("comments")))
dorchester_neighbourhood.toPandas()

NameError: name 'dorchester_neighbourhood' is not defined

Calculate the polarity of each review

In [29]:
dorchester_neighbourhood = dorchester_neighbourhood.withColumn('review_polarity', sentiment_udf(F.col("comments")))
dorchester_neighbourhood.toPandas()

NameError: name 'dorchester_neighbourhood' is not defined

In [30]:
dorchester_neighbourhood_home = dorchester_neighbourhood.where("room_type == 'Entire home/apt'").toPandas()
fig = px.histogram(dorchester_neighbourhood_home, x="review_polarity", color='room_type')
fig.show()

NameError: name 'dorchester_neighbourhood' is not defined

In [31]:
dorchester_neighbourhood_private = dorchester_neighbourhood.where("room_type == 'Private room'").toPandas()
fig = px.histogram(dorchester_neighbourhood_private, x="review_polarity", color='room_type')
fig.show()

NameError: name 'dorchester_neighbourhood' is not defined

In [32]:
dorchester_neighbourhood_shared = dorchester_neighbourhood.where("room_type == 'Shared room'").toPandas()
fig = px.histogram(dorchester_neighbourhood_shared, x="review_polarity", color='room_type')
fig.show()

NameError: name 'dorchester_neighbourhood' is not defined